In [0]:
# Objective of recipe is to:
# Predict on the scm_min_clas_model on the test set
# Get the classification metrics on the test set

In [0]:
library(dataiku)
library(rpart)
library(caret)
library(pROC) # For AUC calculation

In [0]:
# Recipe inputs
folder_path <- dkuManagedFolderPath("xcPrnvPS")
base_test <- dkuReadDataset("base_test", samplingMethod="head", nbRows=100000)


# Construct the full file paths for the models
clas_file_path <- file.path(folder_path, "base_clas_min_model.rds")
wind_file_path  <- file.path(folder_path, "base_wind_model.rds")
rain_file_path  <- file.path(folder_path, "base_rain_model.rds")


# read the .rds model
base_clas_min_model  <- readRDS(file_path)
base_wind_model  <- readRDS(wind_file_path)
base_rain_model  <- readRDS(rain_file_path)


In [0]:
# predicting wind_max and rain & updating the base_test to 

In [0]:
# Recipe outputs
scm_clas_min_model_test <- dkuManagedFolderPath("Xu27U2QF")